In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
import yes24_dbio as ydb
import pandas as pd

In [2]:
# 웹 드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
options = Options()
options.add_experimental_option("detach", True)   # 크롤링이 막히지 않게 해줌
options.add_argument("--disable-blink-features=AutomaticControlled") # 크롤링이 막히지 않게 해줌_자바스크립트 체크 뚫기
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
driver.get("https://www.yes24.com/Product/Category/MonthWeekBestSeller") # 크롤링 할 웹사이트 주소
driver.find_element(By.CSS_SELECTOR, "option[value='120']").click()

In [3]:
all_books = []
for page in range(1, 4):
    time.sleep(8)
    page_html = driver.page_source
    soup = bs(page_html, "lxml")
    book_list = soup.select('#yesBestList > li')
    all_books.append(book_list)
    driver.find_element(By.CSS_SELECTOR, f"a[title='{page+1}']").click()
driver.close()

In [4]:
len(all_books[0])

120

In [5]:
yy = 2023
mon = 8

In [6]:
# 8월 베스트셀러 1-3페이지 리스트
result = pd.DataFrame()
for book_list2 in all_books:
    temp = ydb.extract_bookinfo(yy, mon, book_list2)
    df = pd.DataFrame(temp, columns=['yy', 'mon', 'title_h', 'title_f', 'title_m', 'title_e', 'detail_link', \
                                     'author', 'publisher', 'pub_date', 'price', 'n_reviews',
                                     'review_link', 'rating', 'tags'])
    result = pd.concat([result, df])

# 8월 베스트셀서 1-3페이지 목록의 세부정보 링크로 세부정보 수집
detail_page_data = ydb.detail_page_info(result['detail_link'])
result['book_id'] = result['detail_link'].apply(lambda x: x.split("/")[-1])
final_result_df = pd.merge(result, detail_page_data, how='inner', on="book_id")
display(final_result_df.head(2))
final_result_df.info() 

,yy,mon,title_h,title_f,title_m,title_e,detail_link,author,publisher,pub_date,...,review_link,rating,tags,book_id,page,weight,size,category,book_intro,pub_book_intro
0,2023,8,[도서],,세이노의 가르침,,https://www.yes24.com/Product/Goods/117014613,[세이노(SayNo)],데이원,2023년 03월,...,https://www.yes24.com/Product/Goods/117014613?...,9.1,"[#성공신화, #성공하고싶다면, #삶이고민될때, #인생지침서, #크레마클럽에있어요]",117014613,736쪽,"1,030g",153*224*40mm,"[성공학/경력관리, 처세술/삶의 자세, 국내도서, 자기계발]",\n\r\n재야의 명저 『세이노의 가르침』 2023년판 정식 출간!순자산 천억 원대...,\n\n\r\n재야의 명저 『세이노의 가르침』 2023년판 정식 출간!순자산 천억 ...
1,2023,8,[도서],,1%를 읽는 힘,,https://www.yes24.com/Product/Goods/121812955,[메르],토네이도,2023년 08월,...,https://www.yes24.com/Product/Goods/121812955?...,10.0,"[#이달의굿즈, #이달의주목신간, #얼리리더, #투자가이드, #1분만투자하세요]",121812955,316쪽,512g,145*215*18mm,"[CEO/비즈니스맨, 성공학/경력관리, 국내도서, 경제 경영]","\n\r\n출간 즉시 교보, 예스24, 알라딘 종합 1위네이버 블로그 경제·주식 분...","\n\n\r\n출간 즉시 교보, 예스24, 알라딘 종합 1위네이버 블로그 경제·주식..."


<class 'pandas.core.frame.DataFrame'>
Int64Index: 360 entries, 0 to 359
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   yy              360 non-null    int64 
 1   mon             360 non-null    int64 
 2   title_h         360 non-null    object
 3   title_f         360 non-null    object
 4   title_m         360 non-null    object
 5   title_e         360 non-null    object
 6   detail_link     360 non-null    object
 7   author          360 non-null    object
 8   publisher       360 non-null    object
 9   pub_date        360 non-null    object
 10  price           360 non-null    object
 11  n_reviews       360 non-null    object
 12  review_link     360 non-null    object
 13  rating          360 non-null    object
 14  tags            360 non-null    object
 15  book_id         360 non-null    object
 16  page            360 non-null    object
 17  weight          360 non-null    object
 18  size      

In [7]:
final_result_df.head()

,yy,mon,title_h,title_f,title_m,title_e,detail_link,author,publisher,pub_date,...,review_link,rating,tags,book_id,page,weight,size,category,book_intro,pub_book_intro
0,2023,8,[도서],,세이노의 가르침,,https://www.yes24.com/Product/Goods/117014613,[세이노(SayNo)],데이원,2023년 03월,...,https://www.yes24.com/Product/Goods/117014613?...,9.1,"[#성공신화, #성공하고싶다면, #삶이고민될때, #인생지침서, #크레마클럽에있어요]",117014613,736쪽,"1,030g",153*224*40mm,"[성공학/경력관리, 처세술/삶의 자세, 국내도서, 자기계발]",\n\r\n재야의 명저 『세이노의 가르침』 2023년판 정식 출간!순자산 천억 원대...,\n\n\r\n재야의 명저 『세이노의 가르침』 2023년판 정식 출간!순자산 천억 ...
1,2023,8,[도서],,1%를 읽는 힘,,https://www.yes24.com/Product/Goods/121812955,[메르],토네이도,2023년 08월,...,https://www.yes24.com/Product/Goods/121812955?...,10.0,"[#이달의굿즈, #이달의주목신간, #얼리리더, #투자가이드, #1분만투자하세요]",121812955,316쪽,512g,145*215*18mm,"[CEO/비즈니스맨, 성공학/경력관리, 국내도서, 경제 경영]","\n\r\n출간 즉시 교보, 예스24, 알라딘 종합 1위네이버 블로그 경제·주식 분...","\n\n\r\n출간 즉시 교보, 예스24, 알라딘 종합 1위네이버 블로그 경제·주식..."
2,2023,8,[도서],,아메리칸 프로메테우스 (특별판),로버트 오펜하이머 평전,https://www.yes24.com/Product/Goods/119650815,"[카이 버드, 마틴 셔윈, 최형섭]",사이언스북스,2023년 06월,...,https://www.yes24.com/Product/Goods/119650815?...,9.2,"[#이달의굿즈, #영화드라마원작, #과학_물화생지, #빛나는평전, #과학자]",119650815,"1,056쪽",940g,128*188*38mm,"[자연과학, 과학, 국내도서, 과학자]",\n\r\n2005년 전미 도서 비평가 협회상 수상2006년 퓰리처 상 수상2010...,\n\n\r\n2005년 전미 도서 비평가 협회상 수상2006년 퓰리처 상 수상20...
3,2023,8,[도서],,문과 남자의 과학 공부,나는 무엇이고 왜 존재하며 어디로 가는가?,https://www.yes24.com/Product/Goods/119608737,[유시민],돌베개,2023년 06월,...,https://www.yes24.com/Product/Goods/119608737?...,9.2,"[#이달의굿즈, #재미있는과학, #과학입문, #교양과학]",119608737,304쪽,472g,135*210*20mm,"[자연과학, 과학, 교양으로 읽는 인문, 인문/교양, 국내도서, 인문, 과학 일반,...","\n\r\n과학 공부로 길어 올린 생명과 우주에 관한 진실, 인간과 사회를 이해하는...","\n\n\r\n과학 공부로 길어 올린 생명과 우주에 관한 진실, 인간과 사회를 이해..."
4,2023,8,[도서],,도둑맞은 집중력,"집중력 위기의 시대, 삶의 주도권을 되찾는 법",https://www.yes24.com/Product/Goods/118579613,"[요한 하리, 김하현]",어크로스,2023년 04월,...,https://www.yes24.com/Product/Goods/118579613?...,9.2,"[#이달의굿즈, #집중맞은도둑력, #책읽아웃, #사회비판, #사회의이면]",118579613,464쪽,574g,135*215*23mm,"[관계, 사회 정치, 쉽게 읽는 심리학, 주제로 읽는 심리학, 심리, 국내도서, 인...","\n\r\n아마존, 월 스트리트 저널 올해의 책\r\n애덤 그랜트·수전 케인·힐러리...","\n\n\r\n아마존, 월 스트리트 저널 올해의 책\r\n애덤 그랜트·수전 케인·힐..."


# mysql에 데이터 프레임 저장하기
* pip install pymysql  - python에서 mysql에 접속할 때 필요
* pip install sqlalchemy - sql문을 간결하게 사용하기 위해서 사용 ORM 

In [9]:
import pymysql
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()
import MySQLdb

In [12]:
for col in final_result_df.columns:
    final_result_df[col] = final_result_df[col].apply(str)


In [13]:
engine = create_engine("mysql+mysqldb://root:" + "1234" + "@127.0.0.1:3306/yes24best", encoding='utf-8')
conn = engine.connect()
final_result_df.to_sql(name='best202308', con=engine, if_exists='append', index=False)
conn.close()